# Amazon ECS (Elastic Container Service)

<img src="../_assets/aws_service_icons/ecs.svg" width="80" alt="Amazon ECS">

## Goals
- Understand what **Amazon ECS** is (and how it relates to **Fargate** and **EC2**).
- Know common practical use-cases for ECS.
- See a minimal **AWS SDK** pseudo-code workflow (no execution).


## Prerequisites
- Basic containers knowledge (Docker image, container, ports, env vars).
- Basic AWS networking concepts help (VPC, subnets, security groups).
- Optional: IAM roles/policies.

> This notebook includes **pseudo-code only**. It does not run any AWS SDK calls.


## What ECS is
**Amazon ECS** is AWS’s managed **container orchestration** service: it runs and scales containers for you.

Core ECS concepts:
- **Cluster**: a logical grouping where you run workloads.
- **Task definition**: the blueprint for running containers (image, CPU/memory, env vars, ports, IAM roles, logging).
- **Task**: a running instance of a task definition (one or more containers).
- **Service**: keeps a desired number of tasks running and supports rolling deployments + health checks.

How ECS gets compute:
- **Fargate**: serverless compute for containers (you don’t manage EC2 instances).
- **EC2 launch type**: you run ECS tasks on your own EC2 instances (you manage capacity/patching/AMIs).

Common integrations:
- Container images in **ECR**
- **IAM roles** for tasks (least-privilege access to S3, DynamoDB, etc.)
- Logging to **CloudWatch Logs**
- Load balancing via **ALB/NLB**, and service discovery


## What ECS is practically used for
ECS is commonly used to run containerized workloads in production without running your own orchestrator.

Typical use-cases:
- **Long-running services**: REST/GraphQL APIs, internal services, web apps.
- **Batch jobs**: ETL/ELT, data processing, periodic maintenance tasks.
- **ML workloads**: model inference services, feature generation jobs, evaluation pipelines.

Reasons to choose ECS:
- Tight integration with AWS networking, IAM, and logging.
- **Fargate** option reduces ops burden (no nodes to manage).
- Services support rolling updates, auto-scaling, and load balancers.


## Using ECS with the AWS SDK (pseudo-code)
Below is a minimal, **non-executable** sketch of registering a task definition and running it on ECS using the AWS SDK.

Notes:
- In real projects, use **ECR** for images and avoid hardcoding anything sensitive.
- Prefer **private subnets** + security groups for most workloads.
- Creating ECS resources can incur cost; always plan cleanup.

```python
# PSEUDO-CODE (do not run)

import boto3

region = "us-east-1"
ecs = boto3.client("ecs", region_name=region)

# 1) Create (or reference) a cluster.
cluster_name = "demo-cluster"
ecs.create_cluster(clusterName=cluster_name)

# 2) Register a Fargate-compatible task definition.
task_def = ecs.register_task_definition(
    family="demo-app",
    requiresCompatibilities=["FARGATE"],
    networkMode="awsvpc",
    cpu="256",
    memory="512",
    executionRoleArn="arn:aws:iam::<account-id>:role/ecsTaskExecutionRole",
    taskRoleArn="arn:aws:iam::<account-id>:role/demoAppTaskRole",
    containerDefinitions=[
        {
            "name": "app",
            "image": "<account-id>.dkr.ecr.us-east-1.amazonaws.com/demo:latest",
            "essential": True,
            "portMappings": [{"containerPort": 8080, "protocol": "tcp"}],
            "logConfiguration": {
                "logDriver": "awslogs",
                "options": {
                    "awslogs-group": "/ecs/demo-app",
                    "awslogs-region": region,
                    "awslogs-stream-prefix": "ecs",
                },
            },
        }
    ],
)

task_definition_arn = task_def["taskDefinition"]["taskDefinitionArn"]

# 3) Run a one-off task (common for batch/data jobs).
run = ecs.run_task(
    cluster=cluster_name,
    taskDefinition=task_definition_arn,
    launchType="FARGATE",
    count=1,
    networkConfiguration={
        "awsvpcConfiguration": {
            "subnets": ["subnet-aaaa", "subnet-bbbb"],
            "securityGroups": ["sg-cccccccc"],
            "assignPublicIp": "DISABLED",
        }
    },
)

task_arn = run["tasks"][0]["taskArn"]

# 4) Poll status (logs are typically in CloudWatch Logs).
ecs.describe_tasks(cluster=cluster_name, tasks=[task_arn])

# 5) For long-running services (APIs), you would create an ECS service instead of run_task.
# ecs.create_service(...)
```
